## Preprocessing 🛠️

**Authors**: Carlos Arbonés & Benet Ramió

Before you begin, make sure you have a folder named **data** within this repository containing the following files for the code to run correctly:

- *weather2018.csv*
- *preprocessed-collisions.csv*
- *preprocessed-collisions-2.csv*

In [52]:
import pandas as pd

### Collisions dataset 💥

We assume that the preprocessing conducted in the previous exercise is suitable for this project. Therefore, the only adjustment required is to filter and retain only the rows corresponding to AMBULANCES, TAXIS, and FIRE TRUCKS, as specified in the project description. Additionally, we have chosen to include only the samples from the year 2018 and have removed unnecessary columns for the study.

In [53]:
collisions = pd.read_csv('data/preprocessed-collisions.csv') 
collisions['CRASH_DATETIME'] = pd.to_datetime(collisions['CRASH_DATE'] + ' ' + collisions['CRASH_TIME'], format='%m/%d/%Y %H:%M')
collisions = collisions.drop(columns=['CRASH_DATE', 'CRASH_TIME', 'CONTRIBUTING_FACTOR_VEHICLE2', 'VEHICLE_TYPE_CODE2'])
collisions = collisions[collisions['CRASH_DATETIME'].dt.year == 2018] # Only 2018 data
collisions = collisions[collisions['VEHICLE_TYPE_CODE1'].isin(['Taxi', 'Ambulance', 'Fire truck'])].reset_index(drop=True)

In [54]:
collisions.shape

(4093, 15)

In [55]:
collisions.isna().sum()

BOROUGH                         1385
ZIP_CODE                        1385
LATITUDE                         294
LONGITUDE                        294
TOTAL_INJURED                      1
TOTAL_KILLED                       1
PEDESTRIANS_INJURED                0
PEDESTRIANS_KILLED                 0
CYCLIST_INJURED                    0
CYCLIST_KILLED                     0
MOTORIST_INJURED                   0
MOTORIST_KILLED                    0
CONTRIBUTING_FACTOR_VEHICLE1       0
VEHICLE_TYPE_CODE1                 0
CRASH_DATETIME                     0
dtype: int64

Since we have 1385 missing values in the 'BOROUGH' and 'ZIP_CODE' columns and only 294 missing values in the coordinates, we will impute the values where we have LATITUDE and LONGITUDE but are missing 'BOROUGH' or 'ZIP_CODE'.

In [56]:
# !pip install geopy

In [57]:
from geopy.geocoders import Nominatim

def get_location_info(latitude, longitude):
    geolocator = Nominatim(user_agent="my_geocoder") # initialize geolocator
    location = geolocator.reverse((latitude, longitude), language="en") # reverse geocoding
    borough = location.raw['address']['suburb'].upper() if 'suburb' in location.raw['address'] else location.raw['address']['city'].upper()
    zip_code = location.raw['address']['postcode'] if 'postcode' in location.raw['address'] else None

    return borough, zip_code

In [58]:
no_borough_or_zip = (collisions['BOROUGH'].isna() | collisions['ZIP_CODE'].isna()) # filter for rows with missing borough or zip code
has_lat_long = (collisions['LATITUDE'].notna() & collisions['LONGITUDE'].notna()) # filter for rows with latitude and longitude

missing_locations = collisions[no_borough_or_zip & has_lat_long] 

The following code takes about 10 minutes to execute; we use the previously defined function to impute values for BOROUGH and ZIP CODE. There's no need to run it again as we have already saved the CSV generated from this code.

In [59]:
# import tqdm
# for row in tqdm.tqdm(missing_locations.itertuples()):
#     borough, zip_code = get_location_info(row.LATITUDE, row.LONGITUDE)
#     collisions.loc[row.Index, 'BOROUGH'] = borough
#     collisions.loc[row.Index, 'ZIP_CODE'] = zip_code

In [60]:
collisions['BOROUGH'].value_counts()

BOROUGH
MANHATTAN        1665
BROOKLYN          402
QUEENS            331
BRONX             306
STATEN ISLAND       4
Name: count, dtype: int64

We corrected some inconsistencies by changing neighborhood names from 'THE BRONX' to 'BRONX' and from 'QUEENS COUNTY' to 'QUEENS'. Additionally, we removed the rows that had 'KINGS COUNTY' as the neighborhood since it refers to a neighborhood in California. We assume that the data for those rows is incorrect.

In [61]:
collisions.loc[collisions['BOROUGH'] == 'THE BRONX', 'BOROUGH'] = 'BRONX' # standardize borough names
collisions.loc[collisions['BOROUGH'] == 'QUEENS COUNTY', 'BOROUGH'] = 'QUEENS' # standardize borough names
collisions = collisions[collisions['BOROUGH'] != 'KINGS COUNTY'].reset_index(drop=True) # remove rows with borough 'KINGS COUNTY'

In [62]:
collisions['BOROUGH'].value_counts()

BOROUGH
MANHATTAN        1665
BROOKLYN          402
QUEENS            331
BRONX             306
STATEN ISLAND       4
Name: count, dtype: int64

In [63]:
# percentatge of rows with missing values
collisions.isna().sum() / collisions.shape[0]

BOROUGH                         0.338383
ZIP_CODE                        0.338383
LATITUDE                        0.071830
LONGITUDE                       0.071830
TOTAL_INJURED                   0.000244
TOTAL_KILLED                    0.000244
PEDESTRIANS_INJURED             0.000000
PEDESTRIANS_KILLED              0.000000
CYCLIST_INJURED                 0.000000
CYCLIST_KILLED                  0.000000
MOTORIST_INJURED                0.000000
MOTORIST_KILLED                 0.000000
CONTRIBUTING_FACTOR_VEHICLE1    0.000000
VEHICLE_TYPE_CODE1              0.000000
CRASH_DATETIME                  0.000000
dtype: float64

We can see that the maximum percentage of missing values is in Latitude and Longitude, with 7% of the rows containing null values. To maintain data consistency, we have decided to delete the rows that contain null values. 

In [64]:
collisions = collisions.dropna().reset_index(drop=True) # drop rows with missing values
collisions.shape

(2551, 15)

In [65]:
# collisions.to_csv('data/preprocessed-collisions-2.csv', index=False) 

In [66]:
collisions = pd.read_csv('data/preprocessed-collisions-2.csv') # read preprocessed collisions data

From the "CRASH_DATETIME" column, we derive columns that will be very useful and facilitate the study. Specifically, we create the 'MONTH', 'HOUR', 'DAY_WEEK', 'DAY' columns, which refer to the month, hour, day of the week, and day of the month when the accident occurred, respectively.

In [67]:
collisions['CRASH_DATETIME'] = pd.to_datetime(collisions['CRASH_DATETIME'])
collisions['MONTH'] = collisions['CRASH_DATETIME'].dt.month_name() 
collisions['HOUR'] = collisions['CRASH_DATETIME'].dt.hour
collisions['DAY_WEEK'] = collisions['CRASH_DATETIME'].dt.day_name()
collisions['DAY'] = collisions['CRASH_DATETIME'].dt.day

We derive the column 'CASUALTIES' as the sum of accidents that have had an impact on people's health, i.e., the sum of 'killed' and 'injured'.

In [68]:
collisions['CASUALTIES'] = collisions['TOTAL_INJURED'] + collisions['TOTAL_KILLED']
collisions['CASUALTIES'] = collisions['CASUALTIES'].apply(lambda x: 'Injured or Killed' if x > 0 else 'No Damage')

We make a selection of the columns that are relevant for analysis.

In [69]:
collisions = collisions[['BOROUGH', 'ZIP_CODE', 'LATITUDE', 'LONGITUDE', 
                         'CONTRIBUTING_FACTOR_VEHICLE1', 'VEHICLE_TYPE_CODE1',
                         'CRASH_DATETIME', 'MONTH', 'HOUR', 'DAY_WEEK', 'DAY', 'CASUALTIES']]

### Weather dataset 🌦️

We read the weather dataset, which contains only the data we are interested in, i.e., from June to September 2018. Therefore, there is no need to filter by year. Additionally, for our visualizations and to answer the questions, we will only use the 'icon' column, which contains the type of weather condition for each day. No missing values are observed in this column.

In [70]:
weather = pd.read_csv('data/weather2018.csv')
weather = weather[['datetime', 'icon']] # keep only datetime and icon columns
weather['datetime'] = pd.to_datetime(weather['datetime']) # convert datetime column to datetime type
weather.head()

,datetime,icon
0,2018-06-01,rain
1,2018-06-02,rain
2,2018-06-03,rain
3,2018-06-04,rain
4,2018-06-05,partly-cloudy-day


In [71]:
weather['icon'].value_counts()

icon
rain                 75
clear-day            23
partly-cloudy-day    22
cloudy                2
Name: count, dtype: int64

In [72]:
weather['icon'] = weather['icon'].str.replace('-day', '')

In [73]:
weather.isna().sum()

datetime    0
icon        0
dtype: int64

### Merge datasets 🔄

We merge the collisions dataset with the weather dataset, so for each accident, we now have information about the weather on that day. 

In [74]:
collisions = collisions.merge(weather, how='left', left_on=collisions['CRASH_DATETIME'].dt.date, right_on=weather['datetime'].dt.date)
collisions = collisions.drop(columns=['key_0', 'datetime']) # drop redundant columns
collisions.shape 

(3793, 13)

In [75]:
collisions.head()

,BOROUGH,ZIP_CODE,LATITUDE,LONGITUDE,CONTRIBUTING_FACTOR_VEHICLE1,VEHICLE_TYPE_CODE1,CRASH_DATETIME,MONTH,HOUR,DAY_WEEK,DAY,CASUALTIES,icon
0,BROOKLYN,11218.0,40.644120,-73.98907,Unspecified,Taxi,2018-09-05 23:00:00,September,23,Wednesday,5,No Damage,rain
1,MANHATTAN,10075.0,40.773640,-73.95986,Driver Inattention/Distraction,Taxi,2018-08-05 16:45:00,August,16,Sunday,5,Injured or Killed,clear
2,MANHATTAN,10017.0,40.753480,-73.97879,Unspecified,Taxi,2018-09-18 13:09:00,September,13,Tuesday,18,Injured or Killed,rain
3,MANHATTAN,10022.0,40.757214,-73.97183,Driver Inattention/Distraction,Taxi,2018-09-24 10:05:00,September,10,Monday,24,No Damage,partly-cloudy
4,QUEENS,11101.0,40.752937,-73.92204,Driver Inattention/Distraction,Taxi,2018-09-30 00:50:00,September,0,Sunday,30,No Damage,clear


In [76]:
# collisions.to_csv('data/preprocessed-collisions-final.csv', index=False)